# Intro to EIANN

EIANN is a PyTorch-based library for building neural networks. 

At its core, the syntax for building these networks uses python dicts to specify the network architecture and parameters. We provide two easy ways to generate these dicts: YAML files or a programmatic interface with a NetworkBuilder class. The first method is convenient for quickly prototyping and testing different network architectures. The second method is particularly useful for hyperparameter optimization, systematic architecture search, and experiment reproducibility.
Here we will show you examples of how to do both. 

Since thie library is based on [PyTorch](https://pytorch.org), a basic familiarity with PyTorch objects and syntax is helpful, but not required, to use EIANN. You can geet a quick overview of PyTorch by going through the [PyTorch Quickstart Tutorial](https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html).

### Basic building blocks
To build a neural network using the EIANN library, you need to specify 3 things:
1. The number neural populations within in each "layer" and how many neurons are in each population
2. The projections (how the populations connect to each other)
3. Global parameters that we want to apply to the whole network (e.g. learning rate)

This population-centric approach is flexible and allows you to recurrently connect any two populations, regardless of layer hierarchy.

Each of these things will be defined through 3 separate python dicts. EIANN will then build a PyTorch network the specified architecture.

### 1. Programming interface

In [1]:
import EIANN.EIANN as eiann

/Users/ag1880/github-repos/Milstein-Lab/EIANN/EIANN/utils/data_utils.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
network_config = eiann.NetworkBuilder()

# Define layers and populations
network_config.layer('Input').population('E', size=784)
network_config.layer('H1').population('E', size=500, activation='relu')
network_config.layer('Output').population('E', size=10, activation='softmax')

# Create connections between populations
network_config.connect(source='Input.E', target='H1.E')
network_config.connect(source='H1.E', target='Output.E')

# Build the network
network_config.print_architecture()

Network Architecture:
Input.E (784) -> H1.E (500): No learning rule
H1.E (500) -> Output.E (10): No learning rule



Rather than defining only the weight matrix dimensions, this population-based syntax allows you to also specify attributes at the population level. For example, if you want the network to respect Dale's Law, you can specify whether each population is excitatory or inhibitory. Doing this will automatically bound the sign of outgoing weights, both at initialization and during training. 

In [3]:
network_config.layer('H1').population('E', 500, 'relu').type('Exc') 
network_config.layer('H1').population('SomaI', 50, 'relu').type('Inh') 
network_config.print_architecture()

Network Architecture:
Input.E (784) -> H1.E (500): No learning rule
H1.E (500) -> Output.E (10) [Exc]: No learning rule

Unconnected populations:
H1.SomaI (50) [Inh]



Should you prefer, these weight bounds can also be set or overwritten for individual projections:

In [4]:
network_config.connect(source='H1.E', target='H1.SomaI').type('Exc') 
network_config.connect(source='H1.SomaI', target='H1.E').type('Inh') 
network_config.print_architecture()

Network Architecture:
Input.E (784) -> H1.E (500): No learning rule
H1.SomaI (50) -> H1.E (500) [Inh]: No learning rule
H1.E (500) -> H1.SomaI (50) [Exc]: No learning rule
H1.E (500) -> Output.E (10) [Exc]: No learning rule



So far we have not specified the type of learning rule we want to use. If we want to use a single learning rule (e.g. Backprop) throughout the whole network, this can be done globally:

In [5]:
# Set Backprop as the learning rule for the whole network
network_config.set_learning_rule('Backprop')
network_config.print_architecture()

Network Architecture:
Input.E (784) -> H1.E (500): Backprop
H1.SomaI (50) -> H1.E (500) [Inh]: Backprop
H1.E (500) -> H1.SomaI (50) [Exc]: Backprop
H1.E (500) -> Output.E (10) [Exc]: Backprop



If we instead want a different learning rule for a subset of the connections, we can also manually specify it for a given layer, population, or projection

In [6]:
network_config.connect(source='H1.SomaI', target='H1.E').type('Inh').learning_rule('BCM')
network_config.print_architecture()

Network Architecture:
Input.E (784) -> H1.E (500): Backprop
H1.SomaI (50) -> H1.E (500) [Inh]: BCM
H1.E (500) -> H1.SomaI (50) [Exc]: Backprop
H1.E (500) -> Output.E (10) [Exc]: Backprop



In [7]:
network_config.set_learning_rule_for_population(target_layer='H1', target_population='E', rule='Hebbian', learning_rate=0.01)
network_config.print_architecture()

Network Architecture:
Input.E (784) -> H1.E (500): Hebbian (lr=0.01)
H1.SomaI (50) -> H1.E (500) [Inh]: Hebbian (lr=0.01)
H1.E (500) -> H1.SomaI (50) [Exc]: Backprop
H1.E (500) -> Output.E (10) [Exc]: Backprop



In [8]:
network_config.set_learning_rule_for_layer(target_layer='H1', rule='Ojas_rule', learning_rate=0.01)
network_config.print_architecture()

Network Architecture:
Input.E (784) -> H1.E (500): Ojas_rule (lr=0.01)
H1.SomaI (50) -> H1.E (500) [Inh]: Ojas_rule (lr=0.01)
H1.E (500) -> H1.SomaI (50) [Exc]: Ojas_rule (lr=0.01)
H1.E (500) -> Output.E (10) [Exc]: Backprop



:::{note}
You need to make sure that the learning rule you want to use actually exists in the `EIANN.rules` module. In later tutorials, we will show you how to write your own learning rule.
:::


Once we have specified all the populations and projections that define the network architecture, we can use the `build_network` function to create the neural network (pytorch) object.

In [9]:
network = network_config.build()

Network(
  (criterion): MSELoss()
  (module_dict): ModuleDict(
    (H1E_InputE): Projection(in_features=784, out_features=500, bias=False)
    (H1E_H1SomaI): Projection(in_features=50, out_features=500, bias=False)
    (H1SomaI_H1E): Projection(in_features=500, out_features=50, bias=False)
    (OutputE_H1E): Projection(in_features=500, out_features=10, bias=False)
  )
  (parameter_dict): ParameterDict(
      (H1E_bias): Parameter containing: [torch.FloatTensor of size 500]
      (H1SomaI_bias): Parameter containing: [torch.FloatTensor of size 50]
      (OutputE_bias): Parameter containing: [torch.FloatTensor of size 10]
  )
)


In the next tutorial, we will use EIANN to train a simple feedforward neural network to classify MNIST handwritten digits